In [1]:
import os

In [2]:
%pwd

'd:\\Project\\redwine_end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Project\\redwine_end-to-end'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path :Path
    model_path :Path
    metrics_file_name :Path
    all_param :dict
    target_column : str

In [8]:
from src.constants import *
from src.utils.common import read_yaml_file,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(params_filepath)
        self.schema=read_yaml_file(schema_filepath) 
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluate_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        data_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metrics_file_name=config.metrics_file_name,
            all_param=params,
            target_column=schema.name,
        )
        return data_evaluation_config

In [16]:
from sklearn.metrics import mean_squared_error,r2_score
from src.logging import logger
import os
import pandas as pd
import numpy as np
import pickle
from urllib.parse import urlparse
from utils.common import save_json

In [22]:
class ModelEvaluation:
    def __init__(self,config :ModelEvaluationConfig):
        self.config=config
        
    def model_metric(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_squared_error(actual,pred)
        score=r2_score(actual,pred)
        return rmse,mae,score
    
    def save_file(self):
        
        data_test=pd.read_csv(self.config.test_data_path)
        with open(self.config.model_path,'rb') as f:
            model=pickle.load(f)
            
        
        
        test_X=data_test.drop([self.config.target_column],axis=1)
        test_y=data_test[[self.config.target_column]]
        
        predicted_value=model.predict(test_X)
        
        (rmse,mae,score)= self.model_metric(test_y,predicted_value)
        
        scores={"rmse":rmse,"mae":mae,"score":score}
        save_json(path=Path(self.config.metrics_file_name),data=scores)

In [23]:
try:
    config=ConfigurationManager()
    data_model_config=config.get_model_evaluate_config()
    data_model=ModelEvaluation(config=data_model_config)
    data_model.save_file()
except Exception as e:
    raise e

[2025-02-20 11:58:09,070]: INFO: common:  yaml file config\config.yaml has loaded succesfully
[2025-02-20 11:58:09,070]: INFO: common:  yaml file params.yaml has loaded succesfully
[2025-02-20 11:58:09,073]: INFO: common:  yaml file schema.yaml has loaded succesfully
[2025-02-20 11:58:09,077]: INFO: common:  path has been created in artifacts
[2025-02-20 11:58:09,078]: INFO: common:  path has been created in artifacts/model_evaluation
[2025-02-20 11:58:09,094]: INFO: common:  json file saved to artifacts\model_evaluation\metrics.json
